In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [ ]:
from deeppavlov import configs, build_model

In [ ]:
ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2020-09-30 12:38:30.575 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2020-09-30 12:38:57.174 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


2020-09-30 12:39:03.991 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_rus_bert/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use stan

2020-09-30 12:39:33.971 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_rus_bert/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_rus_bert/model


In [ ]:
from razdel import sentenize
import re
import pandas as pd
import pymorphy2
from nltk.corpus import stopwords
import string
from yargy import Parser, rule, and_, or_
from yargy.predicates import gte, lte, dictionary
from tqdm import tqdm

In [ ]:
file = 'queries_base.xlsx'
xl = pd.ExcelFile(file)
df = xl.parse(xl.sheet_names[0])
del df['Unnamed: 3']
del df['Unnamed: 4']
df.head()

,Текст вопроса,Номер связки\n,Тематика
0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА..."
1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ
2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ
3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ"
4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ"


In [ ]:
def preproccesing(text: str) -> str:
    morph = pymorphy2.MorphAnalyzer()
    # проверяем наличие кириллических символов в слове
    clean_text = ' '.join([word for word in str(text).split() if re.search(r"[а-я]", word)])
    clean_text = ' '.join([word for word in clean_text.lower().split()])
    # убираем знаки препинания
    clean_text = clean_text.translate(str.maketrans(dict.fromkeys(string.punctuation))).split()
    clean_text = [re.sub(r'[«|»|…|\ufeff]', r'', word) for word in clean_text]
    clean_text = [re.sub(r'\n', r' ', word) for word in clean_text]
    # лемматизируем
    clean_text = ' '.join([morph.parse(word)[0].normal_form for word in clean_text])
    # убираем стоп-слова
    clean_text = ' '.join([word for word in clean_text.split() if word not in stopwords.words('russian')])
    return clean_text

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas('PROGRESS>>>')
df['Без удаления NE'] = df['Текст вопроса'].progress_apply(preproccesing)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """Entry point for launching an IPython kernel.


In [ ]:
def preprocess_with_natasha(text: str) -> str:
    segmenter = Segmenter()
    morph_vocab = MorphVocab()

    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)
    ner_tagger = NewsNERTagger(emb)

    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    ner = [word.text for word in doc.spans]
    for n in ner:
        text = text.replace(n, '')

    return text

In [ ]:
for i, el in df.iterrows():
    if type(el['Текст вопроса']) != str:
        df.drop([i], inplace=True)

In [ ]:
df['Natasha'] = df['Текст вопроса'].progress_apply(preprocess_with_natasha)

In [ ]:
df['Natasha'] = df['Natasha'].progress_apply(preproccesing)

In [ ]:
def preprocess_with_deepmipt(text: str, ner_model=ner_model) -> str:
    result = ''
    for sent in sentenize(text):
        ner = ner_model([sent.text])
        indexes = [i for i, el in enumerate(ner[1][0]) if el == 'O']
        res = [el for i, el in enumerate(ner[0][0]) if i in indexes]
        result += ' '.join(res)

    return result

In [ ]:
df['Deepmipt'] = df['Текст вопроса'].progress_apply(preprocess_with_deepmipt)

In [ ]:
df['Deepmipt'] = df['Deepmipt'].progress_apply(preproccesing)

In [ ]:
def preprocess_with_natasha_date(text: str) -> str:
    segmenter = Segmenter()
    morph_vocab = MorphVocab()

    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)
    ner_tagger = NewsNERTagger(emb)

    doc = Doc(text)

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    ner = [word.text for word in doc.spans]
    for n in ner:
        text = text.replace(n, '')

    DAY = and_(
        gte(1),
        lte(31)
    )

    MONTHS = {
        'январь',
        'февраль',
        'март',
        'апрель',
        'мая',
        'июнь',
        'июль',
        'август',
        'сентябрь',
        'октябрь',
        'ноябрь',
        'декабрь',
        'число'
    }

    MONTH_NAME = dictionary(MONTHS)

    DATE = or_(
        rule(
            DAY,
            '-',
            DAY,
            MONTH_NAME
        ),
        rule(
            DAY,
            MONTH_NAME,
        ),
        rule(
            DAY,
            'го',
            MONTH_NAME,
        )
    )
    parser = Parser(DATE)
    for match in parser.findall(text):
        for _ in match.tokens:
            text = text.replace(_.value, '')

    return text

In [ ]:
df['Natasha + new rule'] = df['Текст вопроса'].progress_apply(preprocess_with_natasha_date)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """Entry point for launching an IPython kernel.


In [ ]:
df['Natasha + new rule'] = df['Natasha + new rule'].progress_apply(preproccesing)

In [ ]:
df.to_csv('queries_base_result.csv')

In [ ]:
df.head()

,Unnamed: 0,Текст вопроса,Номер связки\n,Тематика,Без удаления NE,Natasha,Deepmipt,Natasha + new rule
0,0,с уважением Вероника Игоревна Ильич\n\nПосле ...,308.0,"ЗАКРЫТИЕ ГРАНИЦ, ОТКРЫТИЕ ГРАНИЦ РОССИИ И АВИА...",уважение вероника игорь илья август год москва...,уважение август год планировать приехать гость...,уважение август год планировать приехать гость...,уважение год планировать приехать гость сегодн...
1,1,"Здравствуйте! Проинформируйте, пожалуйста, нуж...",324.0,ОРГАНИЗАЦИИ ОТДЫХА ДЕТЕЙ И ИХ ЗДОРОВЛЕНИЯ,здравствовать проинформировать пожалуйста нужн...,здравствовать проинформировать пожалуйста нужн...,здравствовать проинформировать пожалуйста нужн...,здравствовать проинформировать пожалуйста нужн...
2,2,"--\nДобрый день!\n Меня, Сидельникова Андрея...",57.0,БОЛЬНИЧНЫЙ ЛИСТ,добрый день сидельников андрей олег 30071989гр...,добрый день 30071989гра посадить карантин конт...,добрый день 1989г р посадить карантин контактн...,добрый день 30071989гра посадить карантин конт...
3,3,Добрый день.\nВ Кемеровской области согласно п...,45.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",добрый день кемеровский область согласно поста...,добрый день согласно постановление необходимый...,добрый день согласно постановление необходимый...,добрый день согласно постановление необходимый...
4,4,"Здравствуйте, в моем городе Кострома введено о...",3.0,"ШТРАФЫ, НОРМАТИВНЫЕ АКТЫ И РЕКОМЕНДАЦИИ",здравствовать мыть город кострома ввести огран...,здравствовать мыть город ввести ограничение ра...,здравствовать мыть город ввести ограничение ра...,здравствовать мыть город ввести ограничение ра...
